In [237]:
import numpy as np
import pandas as pd
import yfinance as yf
import pypfopt
from pypfopt import expected_returns
from pypfopt import EfficientFrontier
from pypfopt import risk_models
import streamlit as st

In [238]:
tickers = ['TSM','AAPL','AMD','BABA', 'SPY', 'V','QQQ']

In [239]:
data =yf.download(tickers, start='2020-01-01', end='2021-12-31')

[*********************100%***********************]  1 of 1 completed

1 Failed download:
- TSMAAPLAMDBABASPYVQQQ: No data found, symbol may be delisted


In [208]:
data.head()

Adj Close                                                 \
                 AAPL        AMD        BABA         QQQ         SPY   
Date                                                                   
2020-01-02  73.785919  49.099998  219.770004  213.641663  312.395050   
2020-01-03  73.068581  48.599998  217.000000  211.684708  310.029541   
2020-01-06  73.650787  48.389999  216.639999  213.048630  311.212311   
2020-01-07  73.304413  48.250000  217.630005  213.018997  310.337280   
2020-01-08  74.483604  47.830002  218.000000  214.620117  311.991211   

                                       Close                         ...  \
                  TSM           V       AAPL        AMD        BABA  ...   
Date                                                                 ...   
2020-01-02  56.990181  188.125137  75.087502  49.099998  219.770004  ...   
2020-01-03  55.110767  186.628983  74.357498  48.599998  217.000000  ...   
2020-01-06  54.474792  186.225357  74.949997  48.389999  216.639999  ...   
2020-01-07  55.357555  185.733231  74.597504  48.250000  217.630005  ...   
2020-01-08  55.765713  188.912598  75.797501  47.830002  218.000000  ...   

                  Open                            Volume                      \
                   SPY        TSM           V       AAPL       AMD      BABA   
Date                                                                           
2020-01-02  323.540009  59.599998  189.000000  135480400  80331100  15873500   
2020-01-03  321.160004  58.970001  188.410004  146322800  73127400   8604500   
2020-01-06  320.489990  57.599998  188.000000  118387200  47934900  11885500   
2020-01-07  323.019989  57.450001  189.589996  108872000  58061400   9388000   
2020-01-08  322.940002  58.189999  189.490005  132079200  53767000  11959100   

                                                    
                 QQQ       SPY       TSM         V  
Date                                                
2020-01-02  30969400  59151200   8432600   8733000  
2020-01-03  27518900  77709700  10546400   4899700  
2020-01-06  21655300  55653900   8897200  10109500  
2020-01-07  22139300  40496400   7444300   4392300  
2020-01-08  26397300  68296000   5381500   5712000  

[5 rows x 42 columns]

In [209]:
data2 = data['Adj Close']

In [210]:
mu =pypfopt.expected_returns.mean_historical_return(data2)

Index(['AAPL', 'AMD', 'BABA', 'QQQ', 'SPY', 'TSM', 'V'], dtype='object')

In [211]:
cov =pypfopt.risk_models.sample_cov(data2)
w = np.array([.2,.25,.25,.30,.2,.05,.08])

In [212]:
w


array([0.2 , 0.25, 0.25, 0.3 , 0.2 , 0.05, 0.08])

In [227]:
w_min =np.array([-.1,-.2,-.2,-.10,.2,-.05,-.05])

In [228]:
rf =.03

In [229]:
ef = EfficientFrontier(mu, cov,weight_bounds=(-1,1))
ef.add_constraint(lambda x: x <= w)
ef.add_constraint(lambda y: y >= w_min)

In [230]:
sharpe =ef.max_sharpe(risk_free_rate=rf)
clean_weights  = ef.clean_weights()

In [231]:
print(clean_weights)

OrderedDict([('AAPL', 0.2), ('AMD', 0.25), ('BABA', -0.08), ('QQQ', 0.3), ('SPY', 0.2), ('TSM', 0.05), ('V', 0.08)])


In [232]:
ef.portfolio_performance(risk_free_rate= rf, verbose=True)

Expected annual return: 49.6%
Annual volatility: 32.7%
Sharpe Ratio: 1.43


(0.49625479604928757, 0.32698185006133934, 1.4259347910650748)

In [233]:
ef2 = EfficientFrontier(mu, cov,weight_bounds=(-1,1))
ef2.add_constraint(lambda x: x <= w)
ef2.add_constraint(lambda x: x >= w_min)

In [234]:
maxr = ef2.efficient_risk(.3,market_neutral=True)

In [235]:
maxr

OrderedDict([('AAPL', -0.0500000000391206),
             ('AMD', 0.2499999999861249),
             ('BABA', -0.2000000000009475),
             ('QQQ', -0.0999999999865808),
             ('SPY', 0.1999999999987815),
             ('TSM', -0.0499999999586631),
             ('V', -0.0499999999995998)])

In [236]:
ef2.portfolio_performance(risk_free_rate=rf, verbose=True)

Expected annual return: 18.6%
Annual volatility: 12.4%
Sharpe Ratio: 1.26


(0.18623441579207078, 0.1236150677774696, 1.2638784138623145)